In [1]:
import pandas as pd
import imageio
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

from torch.utils.data import Dataset
from torchvision import transforms

import os
import shutil
#from os import join
from time import time
import numpy as np  
import cv2 

import pandas as pd
from time import time

import random

# %run Utils.ipynb
# %run Visualize.ipynb
# %run Model.ipynb

import torch.nn as nn
import torch.nn.functional as F

from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.cm import jet

import matplotlib

matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
%matplotlib inline

from scipy.optimize import minimize
from numpy import linalg as lin
from shapely.geometry import Polygon as P
from shapely.geometry import box

In [2]:
class Dataset(Dataset):

    def __init__(self, root_dir = 'captures', folder="Train/" , kp_file = 'image_%05d_img', transform=None, length=5):
        self.root_dir = os.path.join(path, root_dir, folder)
        
        self.key_pts_file = os.path.join(self.root_dir, kp_file)
        self.transform = transform
        
        files = os.listdir(self.root_dir)
        self.dataLen = int(len(files)/length)

    def __len__(self):
        return self.dataLen - 1

    def __getitem__(self, idx):
        # ignoring the first image as, they are not properly annotated
        idx = idx + 1
        
        image = imageio.imread(os.path.join(self.root_dir, "{}.png".format(self.key_pts_file %idx)))
        if(image.shape[2] == 4):
                image = image[:,:,0:3]
        
        initialKPs = np.array(pd.read_csv(os.path.join(self.root_dir, "{}-ORB.txt".format(self.key_pts_file %idx)), header=None))
        initialKPs = np.c_[ initialKPs, np.ones(initialKPs.shape[0]) ]
        
        KPs = np.array(pd.read_csv(os.path.join(self.root_dir, "{}-GT.txt".format(self.key_pts_file %idx)), header=None))
        KPs = np.c_[ KPs, np.ones(KPs.shape[0]) ]
        
        ## Area of image
        scaleArea = box(0,0,image.shape[0], image.shape[1]).area
        
        image = Image.fromarray(image)
        
        bb = np.array(pd.read_csv(os.path.join(self.root_dir, "{}-BOUND.txt".format(self.key_pts_file %idx)), header=None)).ravel()
        rot = np.array(pd.read_csv(os.path.join(self.root_dir, "{}-Rot.txt".format(self.key_pts_file %idx)), header=None, sep=' '))
        trans = np.array(pd.read_csv(os.path.join(self.root_dir, "{}-Trans.txt".format(self.key_pts_file %idx)), header=None, sep=' ')).reshape((1,3))
        
        item = {'image': image, 'original_image': np.asarray(image) ,'bb': bb, 'initial_keypoints' : initialKPs, 'keypoints': KPs, 'rot':rot, 'trans':trans,'scaleArea':scaleArea}
        if self.transform is not None:
            item = self.transform(item)
        return item
    
class DatasetReal(Dataset):

    def __init__(self, root_dir = 'captures', folder="RealTest/" , kp_file = 'image_%05d_img', transform=None, length=4):
        self.root_dir = os.path.join(path, root_dir, folder)
        
        self.key_pts_file = os.path.join(self.root_dir, kp_file)
        self.transform = transform
        
        files = os.listdir(self.root_dir)
        self.dataLen = int(len(files)/length)

    def __len__(self):
        return self.dataLen

    def __getitem__(self, idx):
        
        image = imageio.imread(os.path.join(self.root_dir, "{}.png".format(self.key_pts_file %idx)))
        if(image.shape[2] == 4):
                image = image[:,:,0:3]
        
        scaleArea = box(0,0,image.shape[0], image.shape[1]).area
        
        image = Image.fromarray(image)

        KPs = np.array(pd.read_csv(os.path.join(self.root_dir, "{}-GT.txt".format(self.key_pts_file %idx)), header=None, sep=' '))
        KPs = np.c_[ KPs, np.ones(KPs.shape[0]) ]
        #print('real KPS' , KPs)
        
        bb = np.array(pd.read_csv(os.path.join(self.root_dir, "{}-BOUND.txt".format(self.key_pts_file %idx)), header=None, sep=' ')).reshape((4))
        #bb = np.array([ [b[0], b[1]], [b[0], b[3]], [b[2], b[1]], [b[2], b[3]] ])
        
        camera = np.array(pd.read_csv(os.path.join(self.root_dir, "{}-Camera.txt".format(self.key_pts_file %idx)), header=None, sep=' ')).reshape((3,3))
        rot = np.array(pd.read_csv(os.path.join(self.root_dir, "{}-Rot.txt".format(self.key_pts_file %idx)), header=None, sep=' ')).reshape((3,3))
        trans = np.array(pd.read_csv(os.path.join(self.root_dir, "{}-Trans.txt".format(self.key_pts_file %idx)), header=None, sep=' ')).reshape((1,3))
        
        item = {'image': image, 'original_image': np.asarray(image), 'initial_keypoints' : KPs.copy(), 'keypoints': KPs, 'bb': bb, 'camera':camera, 'rot':rot, 'trans':trans, 'scaleArea':scaleArea}
        if self.transform is not None:
            item = self.transform(item)
        return item


In [67]:
class CropAndPad:

    def __init__(self, out_size=(256,256), train=True, real=True):
        self.out_size = out_size[::-1]
        self.train = train
        self.real = real

    def __call__(self, sample):
        image, bb = sample['image'], sample['bb']
       # img_size = image.size
        
        if self.real:
            ## This is for Homebrew dataset
            ##min_x,max_y,max_x,min_y = bb[0]-25, bb[1]-25, bb[0] + bb[2] + 25 , bb[1] + bb[3] + 25
            ## For others, probabaly
            min_x,min_y,max_x,max_y = bb[0], bb[1], bb[0] + bb[2], bb[1] + bb[3]
        else:
            min_x,max_y,max_x,min_y = bb[0], bb[1], bb[2], bb[3]

        center_x = (min_x + max_x) / 2
        center_y = (min_y + max_y) / 2
        width, height = max_x-min_x, max_y-min_y
        
        scaleFactor = max([width, height])
        min_x = int(center_x) - int(scaleFactor)//2 
        min_y = int(center_y) - int(scaleFactor)//2
        max_x = int(center_x) + int(scaleFactor)//2 
        max_y = int(center_y) + int(scaleFactor)//2 
        ## This is for Homebrew dataset
        ## Image crop works in a way (0, 0, 10, 10) but here the 
        ## image coordinates are revresed on Y-axis nd so the crop.
        sample['image'] = image.crop(box=(min_x,min_y,max_x,max_y))
        sample['orig_image'] = image
        sample['center'] = np.array([center_x, center_y], dtype=np.float32)
        sample['width'] = width
        sample['height'] = height
        ## This sclae is used for OKS calculation
        sample['scaleArea'] = np.sqrt(np.divide(box(min_x,min_y,max_x,max_y).area, sample['scaleArea']))
        #print(sample['scaleArea'])
        
        w, h= self.out_size
        ## Crop and scale
        sample['crop'] = np.array([min_x, min_y], dtype=np.float32)
        sample['scale'] = np.array([w/scaleFactor, h/scaleFactor] , dtype=np.float32)
        
        if width != self.out_size[0]:
            sample['image'] = sample['image'].resize((w, h))
        if 'mask' in sample:
            sample['mask'] = sample['mask'].crop(box=(min_x,min_y,max_x,max_y)).resize((w, h))
        if 'keypoints' in sample:
            keypoints = sample['keypoints']
            for i in range(keypoints.shape[0]):
                if keypoints[i,0] < min_x or keypoints[i,0] > max_x or keypoints[i,1] < min_y or keypoints[i,1] > max_y:
                    keypoints[i,:] = [0,0,0]
                else:
                    keypoints[i,:2] = (keypoints[i,:2]-sample['crop'] )*sample['scale']
            sample['keypoints'] = keypoints
                
        if 'initial_keypoints' in sample:
            initial_keypoints = sample['initial_keypoints']
            for i in range(initial_keypoints.shape[0]):
                if initial_keypoints[i,0] < min_x or initial_keypoints[i,0] > max_x \
                                or initial_keypoints[i,1] < min_y or initial_keypoints[i,1] > max_y:
                    initial_keypoints[i,:] = [0,0,0]
                else:
                    initial_keypoints[i,:2] = (initial_keypoints[i,:2]-sample['crop'] )*sample['scale']
        
            sample['initial_keypoints'] = initial_keypoints
        sample.pop('bb')
        return sample

    
class ToTensor:

    def __init__(self, downsample_mask=False):
        self.tt = transforms.ToTensor()
        self.downsample_mask=downsample_mask

    def __call__(self, sample):
        sample['image'] = self.tt(sample['image'])
        if 'orig_image' in sample:
            sample['orig_image'] = self.tt(sample['orig_image'])
        if 'mask' in sample:
            if self.downsample_mask:
                sample['mask'] = self.tt(sample['mask'].resize((64,64), Image.ANTIALIAS))
            else:
                sample['mask'] = self.tt(sample['mask'])
        if 'in_mask' in sample:
            sample['in_mask'] = self.tt(sample['in_mask'])
            # sample['in_mask'] = sample['in_mask'].unsqueeze(0)
        if 'keypoint_heatmaps' in sample:
            sample['keypoint_heatmaps'] =\
                torch.from_numpy(sample['keypoint_heatmaps'].astype(np.float32).transpose(2,0,1))
            sample['keypoint_locs'] =\
                torch.from_numpy(sample['keypoint_locs'].astype(np.float32))
            sample['visible_keypoints'] =\
                torch.from_numpy(sample['visible_keypoints'].astype(np.float32))
            
        if 'initial_keypoints_heatmaps' in sample:
            sample['initial_keypoints_heatmaps'] =\
                torch.from_numpy(sample['initial_keypoints_heatmaps'].astype(np.float32).transpose(2,0,1))
            sample['initial_keypoints_locs'] =\
                torch.from_numpy(sample['initial_keypoints_locs'].astype(np.float32))
            sample['visible_initial_keypoints'] =\
                torch.from_numpy(sample['visible_initial_keypoints'].astype(np.float32))
            
        return sample

In [146]:
class prepareRealData():
    
    def __init__(self, source_file_path = '/home/sourabh/Documents/TU-Berlin/Thesis/realdata/HomebrewedData/val',\
                        folder = '000013',\
                        source_file_pattern = '%06d',\
                        src_image_folder = 'rgb',\
                        gt_file = 'scene_gt.json',\
                        gt_info_file = 'scene_gt_info.json',\
                        gt_camera_file = 'scene_camera.json',\
                        object_id = 19,\
                        destination_file_path = '/home/sourabh/Documents/GIT/Python/Deep-NN/PoseTracking/minion/captures',\
                        dest_file_pattern = 'image_%05d_img',\
                        dest_image_folder = 'RealTest'):
        
        print('Processing data of object id : {} !!!!'.format(object_id))
        
        self.source_file_path = os.path.join(source_file_path, folder)
        self.source_file_pattern = source_file_pattern
        self.src_image_folder = src_image_folder
        
        self.gt_file = gt_file
        self.gt_info_file = gt_info_file
        self.gt_camera_file = gt_camera_file
        ## In case of linmod data thi id is the sequence in the list of parameters, in others it is cad model id
        self.object_id = object_id

        self.destination_file_path = destination_file_path
        self.dest_file_pattern = dest_file_pattern
        self.dest_image_folder = dest_image_folder
        self.point3D = pd.read_csv(os.path.join(self.destination_file_path, "GroundTruth/image_groundtruth_img-3DGT.txt"), header=None, sep=',')\
                                    .to_numpy().astype(float)
        
    def createData(self, linmod = False, homebrew = False, tyol=False, ycb=False):

        ## Camera parameters
        ground_truth_camera = pd.read_json(os.path.join(self.source_file_path, self.gt_camera_file))
        print('creating camera parameters...!!')
        for i, r in ground_truth_camera.loc['cam_K'].iteritems():
            #for i, r in row.iteritems():
            if type(r) is list:
                np.savetxt(os.path.join(self.destination_file_path, self.dest_image_folder, '{}-Camera.txt'.format(self.dest_file_pattern %i)), np.array([r]), fmt='%s')
        
        ## Ground truth parameters
        print('creating ground-truth parameters...!!')
        
        if tyol or ycb:
            print('Creating {}'.format('tyol' if  tyol else 'ycb'))
            
            ground_truth = pd.read_json(os.path.join(self.source_file_path, self.gt_file), orient='index')
            for i, r in ground_truth.iterrows():
                r = r[0]
                
                rot = np.array([r['cam_R_m2c']]).reshape((3,3))
                trans = np.array([r['cam_t_m2c']]).reshape((3,1))
                #trans[1] = -trans[1]
                
                np.savetxt(os.path.join(self.destination_file_path, self.dest_image_folder, '{}-Rot.txt'.format(self.dest_file_pattern %i)), rot, fmt='%s')
                np.savetxt(os.path.join(self.destination_file_path, self.dest_image_folder, '{}-Trans.txt'.format(self.dest_file_pattern %i)), trans, fmt='%s')


                shutil.copy(os.path.join(self.source_file_path, self.src_image_folder, '{}.png'.format(self.source_file_pattern %i)), \
                                    os.path.join(self.destination_file_path, self.dest_image_folder, '{}.png'.format(self.dest_file_pattern %i)))
                
                
                

                cameraMatrix = np.array(pd.read_csv(os.path.join(self.destination_file_path, self.dest_image_folder, \
                                                                    "{}-Camera.txt".format(self.dest_file_pattern %i)), header=None, sep=' '))
                cameraMatrix = cameraMatrix.reshape((3, 3))
                GTPointsInit = np.matmul(cameraMatrix, np.matmul(rot, np.squeeze(self.point3D).T) + trans).T
                point2d = GTPointsInit[:,:2] / GTPointsInit[:,2, None]


                np.savetxt(os.path.join(self.destination_file_path, self.dest_image_folder, '{}-GT.txt'.format(self.dest_file_pattern %i)), np.array(point2d), fmt='%s')
            
            ground_truth = pd.read_json(os.path.join(self.source_file_path, self.gt_info_file), orient='index')
            for i, r in ground_truth.iterrows():
                r = r[0]
                np.savetxt(os.path.join(self.destination_file_path, self.dest_image_folder, '{}-BOUND.txt'.format(self.dest_file_pattern %i)), np.array([r['bbox_obj']]), fmt='%s')
            
            
        if linmod: 
            print('Creating linmod')
            
            ground_truth = pd.read_json(os.path.join(self.source_file_path, self.gt_file), orient='index')
            for i, r in ground_truth.iterrows():
                r = r[self.object_id]
                np.savetxt(os.path.join(self.destination_file_path, self.dest_image_folder, '{}-Rot.txt'.format(self.dest_file_pattern %i)), np.array([r['cam_R_m2c']]), fmt='%s')
                np.savetxt(os.path.join(self.destination_file_path, self.dest_image_folder, '{}-Trans.txt'.format(self.dest_file_pattern %i)), np.array([r['cam_t_m2c']]), fmt='%s')
                
                shutil.copy(os.path.join(self.source_file_path, self.src_image_folder, '{}.png'.format(self.source_file_pattern %i)), \
                                    os.path.join(self.destination_file_path, self.dest_image_folder, '{}.png'.format(self.dest_file_pattern %i)))
                
            ground_truth = pd.read_json(os.path.join(self.source_file_path, self.gt_info_file), orient='index')
            for i, r in ground_truth.iterrows():
                r = r[self.object_id]
                np.savetxt(os.path.join(self.destination_file_path, self.dest_image_folder, '{}-BOUND.txt'.format(self.dest_file_pattern %i)), np.array([r['bbox_obj']]), fmt='%s')
          
            
        
        if homebrew:
            print('Creating Homebrew')
            
            ground_truth = pd.read_json(os.path.join(self.source_file_path, self.gt_file))
            for index, row in ground_truth.iterrows():
                for i, r in row.iteritems():
                    if r['obj_id'] == self.object_id:
                        
                        rot = np.array([r['cam_R_m2c']]).reshape((3,3))
                        trans = np.array([r['cam_t_m2c']]).reshape((3,1))
                        
                        np.savetxt(os.path.join(self.destination_file_path, self.dest_image_folder, '{}-BOUND.txt'.format(self.dest_file_pattern %i)), np.array([r['obj_bb']]), fmt='%s')
                        np.savetxt(os.path.join(self.destination_file_path, self.dest_image_folder, '{}-Rot.txt'.format(self.dest_file_pattern %i)), rot, fmt='%s')
                        np.savetxt(os.path.join(self.destination_file_path, self.dest_image_folder, '{}-Trans.txt'.format(self.dest_file_pattern %i)), trans, fmt='%s')

                        cameraMatrix = np.array(pd.read_csv(os.path.join(self.destination_file_path, self.dest_image_folder, \
                                                                            "{}-Camera.txt".format(self.dest_file_pattern %i)), header=None, sep=' '))
                        cameraMatrix = cameraMatrix.reshape((3, 3))
                        # Only when mesh is ot alligned.
                        rot[:,0] = -rot[:,0]
                        GTPointsInit = np.matmul(cameraMatrix, np.matmul(rot, np.squeeze(self.point3D).T) + trans).T
                        point2d = GTPointsInit[:,:2] / GTPointsInit[:,2, None]
                        
                        
                        np.savetxt(os.path.join(self.destination_file_path, self.dest_image_folder, '{}-GT.txt'.format(self.dest_file_pattern %i)), np.array(point2d), fmt='%s')
                        
                        shutil.copy(os.path.join(self.source_file_path, self.src_image_folder, '{}.png'.format(self.source_file_pattern %i)), \
                                    os.path.join(self.destination_file_path, self.dest_image_folder, '{}.png'.format(self.dest_file_pattern %i)))

                        
                        
    def visualizeRealData(self, step= 5, length = 5):

        dataset = DatasetReal(length=length)
        composed = transforms.Compose([CropAndPad(out_size=(256, 256), real=True),ToTensor()])
        lenData = len(dataset)

        for test_num in range(0, lenData, step):
            
            fig = plt.figure()
            for i in range(0, step):
                ax = plt.subplot(1, step , i + 1)
                
                transformed_sample = composed(dataset[test_num + i])
                plt.tight_layout()
                ax.set_title(test_num + i)

                image = transformed_sample['image'].data   # get the image from it's Variable wrapper
                image = image.numpy()   # convert to numpy array from a Tensor
                image = np.transpose(image, (1, 2, 0)) 

                plt.imshow(image)

            plt.show()
            
    def visualizeSynthData(self, step= 5, length = 10):

        dataset = Dataset(root_dir = 'captures', folder="SynthTest/", length=length)
        composed = transforms.Compose([CropAndPad(out_size=(256, 256), real=False),ToTensor()])
        lenData = len(dataset)

        for test_num in range(0, lenData, step):
            
            fig = plt.figure()
            for i in range(0, step):
                ax = plt.subplot(1, step , i + 1)
                
                transformed_sample = composed(dataset[test_num + i])
                plt.tight_layout()
                ax.set_title(test_num + i)

                image = transformed_sample['image'].data   # get the image from it's Variable wrapper
                image = image.numpy()   # convert to numpy array from a Tensor
                image = np.transpose(image, (1, 2, 0)) 

                plt.imshow(image)

            plt.show()
    
    
    def refinedData(self, numbers):
        
        if type(numbers) is not list:
            print('please pass a list of numbers')
            return
        print('refine data' ,  os.path.join(self.destination_file_path, self.dest_image_folder))
        
        for (dirpath, dirnames, filenames) in os.walk(os.path.join(self.destination_file_path, self.dest_image_folder)):
            print(dirpath)
            if dirpath == os.path.join(self.destination_file_path, self.dest_image_folder):
                
                count = 0
                renameCount = 0
                os.chdir(dirpath)
                
                filenames.sort()
                for filename in filenames:
                    if filename.endswith('png'):
                        file = filename.split('.')[0]

                        if count in numbers:
                            os.rename('{}.png'.format(file), '{}.png'.format(self.dest_file_pattern %renameCount))
                            os.rename('{}-BOUND.txt'.format(file), '{}-BOUND.txt'.format(self.dest_file_pattern %renameCount))
                            os.rename('{}-Rot.txt'.format(file), '{}-Rot.txt'.format(self.dest_file_pattern %renameCount))
                            os.rename('{}-Trans.txt'.format(file), '{}-Trans.txt'.format(self.dest_file_pattern %renameCount))
                            os.rename('{}-Camera.txt'.format(file), '{}-Camera.txt'.format(self.dest_file_pattern %renameCount))
                            os.rename('{}-GT.txt'.format(file), '{}-GT.txt'.format(self.dest_file_pattern %renameCount))

                            print('file {} renamed to count number {}'.format(file, self.dest_file_pattern %renameCount))
                            renameCount += 1

                        else:
                            os.remove('{}.png'.format(file))
                            os.remove('{}-BOUND.txt'.format(file))
                            os.remove('{}-Rot.txt'.format(file))
                            os.remove('{}-Trans.txt'.format(file))
                            os.remove('{}-Camera.txt'.format(file))
                            os.remove('{}-GT.txt'.format(file))

                            print('file {} removed '.format(file))

                        count += 1
        
        

In [147]:
linmod =False
homebrew = False
tyol = False
ycb=False

### Homebrew dataset
# path = "/home/sourabh/Documents/GIT/Python/Deep-NN/PoseTracking/Real/Buffalo/1000/captures_buffalo_1000_random"
# homebrew =True

# data = prepareRealData(object_id = 4, folder = '000003', destination_file_path = os.path.join(path, 'captures'))


#### Linmod dataset
# path = "/home/sourabh/Documents/GIT/Python/Deep-NN/PoseTracking/Real/Floater/500"
# linmod = True

# data = prepareRealData(source_file_path = '/home/sourabh/Documents/TU-Berlin/Thesis/realdata/linmod/lmo_test_all/test',\
#                         folder = '000002',\
#                         object_id = 4,\
#                         source_file_pattern = '%06d',\
#                         destination_file_path = os.path.join(path, 'captures'))

#### TYOL dataset
# path = "/home/sourabh/Documents/GIT/Python/Deep-NN/PoseTracking/Real/Floater/500"
# tyol = True

# data = prepareRealData(source_file_path = '/home/sourabh/Documents/TU-Berlin/Thesis/realdata/TYOLData/val/test',\
#                         folder = '000003',\
#                         object_id = 4,\
#                         source_file_pattern = '%06d',\
#                         destination_file_path = os.path.join(path, 'captures'))

#### YCB dataset
path = "/home/sourabh/Documents/GIT/Python/Deep-NN/PoseTracking/Real/Can/1000/captures_can_1000_random"
ycb = True

data = prepareRealData(source_file_path = '/home/sourabh/Documents/TU-Berlin/Thesis/realdata/Ycb/ycbv_test_all/test',\
                        folder = '000048',\
                        object_id = 1,\
                        source_file_pattern = '%06d',\
                        destination_file_path = os.path.join(path, 'captures'))

if homebrew:
    linmod =False
    tyol = False
    ycb=False
elif linmod:
    homebrew =False
    tyol = False
    ycb=False
elif tyol:
    linmod =False
    homebrew = False
    ycb=False
elif ycb:
    linmod =False
    homebrew = False
    tyol=False

Processing data of object id : 1 !!!!


In [148]:
## %%%% Use this section to create real data %%%#####

data.createData(linmod=linmod, homebrew=homebrew, tyol=tyol, ycb=ycb)
#data.visualizeRealData(step=4, length=6)
#data.visualizeSynthData(step=4, length=10)

imageList = list(np.arange(10, 40))
imageList.extend(list(np.arange(310, 340)))
imageList.extend(list(np.arange(810, 840)))
imageList.extend(list(np.arange(1280, 1290)))
print(imageList)
data.refinedData(imageList)

#data.refinedData(list(np.arange(0, 340, 6)))
#data.refinedData([3,4,5,6,7,8,9,17,18,19,20,21,22,23])
#data.refinedData(random.sample(range(80), 50))

creating camera parameters...!!
creating ground-truth parameters...!!
Creating ycb
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289]
refine data /home/sourabh/Documents/GIT/Python/Deep-NN/PoseTracking/Real/Can/1000/captures_can_1000_random/captures/RealTest
/home/sourabh/Documents/GIT/Python/Deep-NN/PoseTracking/Real/Can/1000/captures_can_1000_random/captures/RealTest
file image_00000_img removed 
file image_00001_img removed 
file image_00002_img removed 
file image_00003_img removed 
file image_00004_img removed 
file image_00005_img removed 
file image_00006_i